**Dependencies**

In [ ]:
import pandas as pd 
import numpy as np
import sys
import importlib

from tensorflow import keras
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.metrics import confusion_matrix

sys.path.insert(0, '../functions/')

import preprocess_data
import train_test_models

**Data Preprocessing**

In [ ]:
importlib.reload(preprocess_data)

timeframe = "1h"
data = pd.read_csv(f"../data/BTC_EUR-{timeframe}.csv")
data['Timestamp'] = pd.to_datetime(data['Timestamp'], unit='ms')
data.set_index(keys='Timestamp', inplace=True)

shift_days = 10
window = 24
value_to_predict = 'Close'

train_x, val_x, test_x, train_y, val_y, test_y = preprocess_data.preprocess_data(data, k=shift_days, column=value_to_predict, window=window)

print('Train shape: {}\nTest shape: {}\nBalanced Train: {}'
    .format(train_x.shape, test_x.shape, train_y.sum() == train_x.shape[0]/2))

**Network Training**

In [ ]:
importlib.reload(train_test_models)

num_nets = 3

model = list()
for i in range(num_nets):
    model.append(train_test_models.train_model(net=i, 
                                        train_data=(train_x, train_y), 
                                        batch_size=128, 
                                        epochs=1, 
                                        loss='binary_crossentropy',
                                        verbose=0))

In [12]:
importlib.reload(train_test_models)

num_nets = 3

model = list()

_, history = train_test_models.train_model(net=i, 
                                        train_data=(train_x, train_y), 
                                        batch_size=128, 
                                        epochs=1, 
                                        loss='binary_crossentropy',
                                        verbose=0, return_history=True)
                            
history

**Network Testing**

In [ ]:
for i in range(num_nets, only_accuracy=False):
    train_test_models.test_model(model[i], test_x, test_y)

**Feature Extractors**

In [ ]:
features = list()
features_x = list()
for i in range(num_nets):
    features.append(keras.Model(
        inputs=model[i].inputs, 
        outputs=model[i].layers[-4].output
    ))
    features_x.append(features[i](train_x).numpy())

features_conc = np.concatenate((features_x), axis=1)
features_conc.shape

**SVM Training**

In [ ]:
print("Fitting the classifier to the training set")
param_grid = {'C': [0.01, 0.1, 1, 10, 100], 'kernel': ['rbf', 'linear','sigmoid']}
classifier = GridSearchCV(svm.SVC(class_weight='balanced'), param_grid)
classifier = classifier.fit(features_conc, train_y[:,0])
print("Best estimator found by grid search:")
print(classifier.best_estimator_)

**SVM Testing**

In [ ]:
test_features = list()

for i in range(num_nets):
    test_features.append(features[i](test_x).numpy())

test_con = np.concatenate((test_features), axis =  1)

y_hat = classifier.predict(test_con)

buys = test_y.sum()
holds = len(test_y)-test_y.sum()

cm = confusion_matrix(test_y, y_hat)
FP = cm.sum(axis=0) - np.diag(cm)  
FN = cm.sum(axis=1) - np.diag(cm)
TP = np.diag(cm)
TN = cm.sum() - (FP + FN + TP)


print('Buys: {:d}, Holds: {:d}'.format(buys, holds))
print("True Buys: {:d}, False Buys: {:d}, True Holds: {:d}, False Holds: {:d}".format(TP[0],FP[0],TN[0],FN[0]))
print("Accuracy: {:.4f}".format(np.count_nonzero(((y_hat == test_y[:,0])))/len(test_y)))

In [ ]:
np.save("../data/train_x_15m.npy", train_x)
np.save("../data/train_y_15m.npy", train_y)
np.save("../data/test_x_15m.npy", test_x)
np.save("../data/test_y_15m.npy", test_y)